In [1]:
import sys
sys.path.append("..")

In [2]:
import torch
import numpy as np
import random
import torch.optim as optim

from algorithms.algorithms import *
from utils.models import *
from utils.dataloaders import *

from tabulate import tabulate

In [3]:
device = 'cpu'
np.random.seed(0)

dataset = CreditCardClientsDataset(device=device, sensitive_feature_labels=["SEX", "AGE"])

In [4]:
def lr_kde_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_kde_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'], 
            h=hp['h'], 
            delta_huber=hp['delta_huber'], 
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def lr_fb_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fb_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def lr_fc_model_runner(dataset, hp, seeds):
    test = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    train = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}
    
    val = {'accuracy':[],
            'ei_disparity':[],
            'dp_disparity':[],
            'eo_disparity':[],
            'eodd_disparity':[]}

    def append_res(l,acc,ei,dp,eo,eodd):
        l['accuracy'].append(acc)
        l['ei_disparity'].append(ei)
        l['dp_disparity'].append(dp)
        l['eo_disparity'].append(eo)
        l['eodd_disparity'].append(eodd)

    for i in range(len(seeds)):
        print('training seed', seeds[i] ,'started')
        random.seed(seeds[i])
        np.random.seed(seeds[i])
        torch.manual_seed(seeds[i]) 

        model = logReg(num_features=dataset.XZ_train.shape[1])
        model = model.to(device)
        
        lr = hp['learning_rate']
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
        
        results = trainer_fc_fair(
            model,
            dataset,
            optimizer,
            device,
            n_epochs=hp['n_epochs'],
            batch_size=hp['batch_size'], 
            z_blind=False,
            fairness=hp['fairness'], 
            lambda_=hp['lambda_'],
            optimal_effort=True, 
            delta_effort=hp['delta_effort']
            )
        
        append_res(train,results.train_acc_hist[-1],results.train_ei_hist[-1],results.train_dp_hist[-1],results.train_eo_hist[-1],results.train_eodd_hist[-1])
        append_res(val,results.val_acc,results.val_ei,results.val_dp,results.val_eo,results.val_eodd)
        append_res(test,results.test_acc,results.test_ei,results.test_dp,results.test_eo,results.test_eodd)

    def get_res(l):
        res = {}
        res['accuracy_mean'] = np.mean(l['accuracy'])
        res['accuracy_var'] = np.std(l['accuracy'])
        res['accuracy_list'] = l['accuracy']
        res['ei_mean'] = np.mean(l['ei_disparity'])
        res['ei_var'] = np.std(l['ei_disparity'])
        res['ei_list'] = l['ei_disparity']
        res['dp_mean'] = np.mean(l['dp_disparity'])
        res['dp_var'] = np.std(l['dp_disparity'])
        res['dp_list'] = l['dp_disparity']
        res['eo_mean'] = np.mean(l['eo_disparity'])
        res['eo_var'] = np.std(l['eo_disparity'])
        res['eo_list'] = l['eo_disparity']
        res['eodd_mean'] = np.mean(l['eodd_disparity'])
        res['eodd_var'] = np.std(l['eodd_disparity'])
        res['eodd_list'] = l['eodd_disparity']
        return res

    res_train = get_res(train)
    res_val = get_res(val)
    res_test = get_res(test)
    print('Training finished for all seeds.')
    
    return res_train, res_val, res_test

def experiment_runner(dataset, SGD_hp, EI_hp_fc, EI_hp_kde, EI_hp_fb, seeds):
    
    _, _, SGD = lr_kde_model_runner(dataset, SGD_hp, seeds)
    _, _, EI_fc = lr_fc_model_runner(dataset, EI_hp_fc, seeds)
    _, _, EI_kde = lr_kde_model_runner(dataset, EI_hp_kde, seeds)
    _, _, EI_fb = lr_fb_model_runner(dataset, EI_hp_fb, seeds)
    
    return SGD, EI_fc, EI_kde, EI_fb

def fb_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fb_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

def kde_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_kde_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

def fc_hyperparameter_test(dataset, hp_test, seed=0):
    hp = hp_test.copy()
    result = []
    for i in hp_test['learning_rate']:
        for k in hp_test['lambda_']:
            c = []
            hp['learning_rate'] = i
            hp['lambda_'] = k
            train, val, _ = lr_fc_model_runner(dataset, hp, seeds=[seed])
            c.append(hp['learning_rate'])
            c.append(hp['lambda_'])
            c.append(train['accuracy_mean'])
            c.append(val['accuracy_mean'])
            c.append(val['ei_mean'])
            c.append(val['dp_mean'])
            c.append(val['eo_mean'])
            c.append(val['eodd_mean'])
            result.append(c)
    print(tabulate(result, headers=['learning_rate', 'lambda_', 'accuracy_train', 'accuracy_val','ei', 'dp', 'eo', 'eodd']))

In [5]:
SGD_hp_test = {}
SGD_hp_test['learning_rate'] = [0.0001, 0.001, 0.01, 0.1]
SGD_hp_test['lambda_'] = [0]
SGD_hp_test['n_epochs'] = 100
SGD_hp_test['batch_size'] = 1024 
SGD_hp_test['fairness'] = ''
SGD_hp_test['h'] = 0.01
SGD_hp_test['delta_huber'] = 0.5
SGD_hp_test['delta_effort'] = 3.1

In [6]:
kde_hyperparameter_test(dataset, SGD_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:22<00:00,  4.46epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:22<00:00,  4.50epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:21<00:00,  4.55epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:21<00:00,  4.66epochs/s]


Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val         ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ---------  ---------  ---------  ---------
         0.0001          0          0.801058        0.798307  0.0283603  0.0150492  0.036735   0.036735
         0.001           0          0.814497        0.814603  0.0136013  0.0309424  0.0683472  0.0683472
         0.01            0          0.817302        0.816085  0.0075992  0.0227132  0.04639    0.04639
         0.1             0          0.81836         0.818624  0.0171846  0.0242504  0.0497741  0.0497741


In [7]:
SGD_hp = SGD_hp_test.copy()
SGD_hp['learning_rate'] = 0.01
SGD_hp['lambda_'] = 0

In [8]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.01]
EI_hp_test['lambda_'] = [0, 0.2, 0.6, 0.8, 0.9]
EI_hp_test['fairness'] = 'EI'

In [9]:
kde_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training:   0%|          | 0/100 [00:00<?, ?epochs/s]

Training: 100%|██████████| 100/100 [00:31<00:00,  3.19epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:28<00:00,  3.48epochs/s]


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:29<00:00,  3.36epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:29<00:00,  3.39epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ----------  ---------  ---------  ---------
           0.01        0            0.817302        0.816085  0.0075992   0.0227132  0.04639    0.04639
           0.01        0.2          0.817354        0.815873  0.00497947  0.0223926  0.0473352  0.0473352
           0.01        0.6          0.817196        0.81672   0.00453257  0.0224305  0.0482804  0.0482804
           0.01        0.8          0.816455        0.815661  0.00281881  0.0216503  0.0487225  0.0487225
           0.01        0.9          0.814974        0.813333  0.0105269   0.0392893  0.081888   0.081888


In [10]:
EI_hp_kde = EI_hp_test.copy()
EI_hp_kde['learning_rate'] = 0.01
EI_hp_kde['lambda_'] = 0.8

In [11]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.01]
EI_hp_test['lambda_'] = [0, 0.6, 0.8, 0.9, 0.95]
EI_hp_test['fairness'] = 'EI'

In [12]:
fc_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:22<00:00,  4.47epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:21<00:00,  4.64epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:22<00:00,  4.42epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:21<00:00,  4.75epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val          ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  ----------  ---------  ---------  ---------
           0.01       0             0.817302        0.816085  0.0075992   0.0227132  0.04639    0.04639
           0.01       0.6           0.817407        0.816296  0.00560335  0.0217515  0.0473352  0.0473352
           0.01       0.8           0.816984        0.81672   0.00663469  0.0215337  0.0475754  0.0475754
           0.01       0.9           0.816349        0.816296  0.00830017  0.0205195  0.0514107  0.0514107
           0.01       0.95          0.815767        0.816296  0.00430367  0.0192697  0.049815   0.049815


In [13]:
EI_hp_fc = EI_hp_test.copy()
EI_hp_fc['learning_rate'] = 0.01
EI_hp_fc['lambda_'] = 0.95

In [16]:
EI_hp_test = SGD_hp_test.copy()
EI_hp_test['learning_rate'] = [0.01]
EI_hp_test['lambda_'] = [0, 0.2, 0.4, 0.6, 0.8]
EI_hp_test['fairness'] = 'EI'

In [17]:
fb_hyperparameter_test(dataset, EI_hp_test, seed=0)

training seed 0 started


Training: 100%|██████████| 100/100 [00:28<00:00,  3.48epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:27<00:00,  3.66epochs/s]


Training finished for all seeds.
training seed 0 started


...

Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:27<00:00,  3.64epochs/s]


Training finished for all seeds.
training seed 0 started


Training: 100%|██████████| 100/100 [00:30<00:00,  3.23epochs/s]

Training finished for all seeds.
  learning_rate    lambda_    accuracy_train    accuracy_val           ei         dp         eo       eodd
---------------  ---------  ----------------  --------------  -----------  ---------  ---------  ---------
           0.01        0            0.817302        0.816085  0.0075992    0.0227132  0.04639    0.04639
           0.01        0.2          0.818466        0.818413  0.00146378   0.0210355  0.0504655  0.0504655
           0.01        0.4          0.817989        0.818413  0.0126329    0.0184246  0.0395654  0.0395654
           0.01        0.6          0.817884        0.81672   0.0148132    0.0196945  0.0375956  0.0375956
           0.01        0.8          0.808307        0.805926  0.000942176  0.0221216  0.0354855  0.0354855


In [18]:
EI_hp_fb = EI_hp_test.copy()
EI_hp_fb['learning_rate'] = 0.01
EI_hp_fb['lambda_'] = 0.8

In [19]:
seeds = np.arange(1,6)

SGD, EI_fc, EI_kde, EI_fb = experiment_runner(dataset, SGD_hp, EI_hp_fc, EI_hp_kde, EI_hp_fb, seeds)

/var/folders/_1/grmv219n3mzbyfj882m03j3c0000gn/T/ipykernel_3479/3561611485.py:29: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(seeds[i])


training seed 1 started


Training:   0%|          | 0/100 [00:00<?, ?epochs/s]

Training: 100%|██████████| 100/100 [00:23<00:00,  4.34epochs/s]


training seed 2 started


...

training seed 4 started


Training: 100%|██████████| 100/100 [00:27<00:00,  3.67epochs/s]


training seed 5 started


Training: 100%|██████████| 100/100 [00:27<00:00,  3.63epochs/s]

Training finished for all seeds.


In [20]:
result = []
models = ["SGD", "EI FC", "EI KDE", "EI FB"]
sol = [SGD, EI_fc, EI_kde, EI_fb]
for i in range(len(models)):
    c = []
    c.append(models[i])
    res = sol[i]
    c.append(res['accuracy_mean'])
    c.append(res['accuracy_var'])
    c.append(res['ei_mean'])
    c.append(res['ei_var'])
    c.append(res['dp_mean'])
    c.append(res['dp_var'])
    c.append(res['eo_mean'])
    c.append(res['eo_var'])
    c.append(res['eodd_mean'])
    c.append(res['eodd_var'])
    result.append(c)

print(tabulate(result, headers=["model","accuracy_mean","accuracy_var","ei_mean","ei_var", "dp_mean", "dp_var","eo_mean","eo_var","eodd_mean","eodd_var"]))

model      accuracy_mean    accuracy_var     ei_mean       ei_var    dp_mean       dp_var    eo_mean      eo_var    eodd_mean    eodd_var
-------  ---------------  --------------  ----------  -----------  ---------  -----------  ---------  ----------  -----------  ----------
SGD             0.825597     0.000590337  0.0133685   0.00185294   0.0204723  0.000718165  0.0261748  0.00162436    0.0261748  0.00162436
EI FC           0.823328     0.00076162   0.00334426  0.00101703   0.0168162  0.000819615  0.016367   0.00204342    0.016367   0.00204342
EI KDE          0.824953     0.000710214  0.00283053  0.000363106  0.0180179  0.00115796   0.020215   0.00307935    0.020215   0.00307935
EI FB           0.809142     0.00293337   0.00328821  0.00154129   0.0122883  0.00290608   0.0225487  0.010165      0.024597   0.00688776
